# Getting Started with Julia in Colab/Jupyter
You can either run this notebook in Google Colab, or using Jupyter on your own machine.

## Running on Google Colab
1. Work on a copy of this notebook: _File_ > _Save a copy in Drive_ (you will need a Google account). Alternatively, you can download the notebook using _File_ > _Download .ipynb_, then upload it to [Colab](https://colab.research.google.com/).
2. Execute the following cell (click on it and press Ctrl+Enter) to install Julia, IJulia (the Jupyter kernel for Julia) and other packages. You can update `JULIA_VERSION` and the other parameters, if you know what you're doing. Installation takes 2-3 minutes.
3. Reload this page (press Ctrl+R, or ⌘+R, or the F5 key) and continue to the _Checking the Installation_ section.

* _Note_: If your Colab Runtime gets reset (e.g., due to inactivity), repeat steps 2 and 3.

In [ ]:
%%shell
set -e

#---------------------------------------------------#
JULIA_VERSION="1.6.0" # any version ≥ 0.7.0
JULIA_PACKAGES="IJulia Distances"
JULIA_PACKAGES_IF_GPU="CUDA"
JULIA_NUM_THREADS=4
#---------------------------------------------------#

if [ -n "$COLAB_GPU" ] && [ -z `which julia` ]; then
  # Install Julia
  JULIA_VER=`cut -d '.' -f -2 <<< "$JULIA_VERSION"`
  echo "Installing Julia $JULIA_VERSION on the current Colab Runtime..."
  BASE_URL="https://julialang-s3.julialang.org/bin/linux/x64"
  URL="$BASE_URL/$JULIA_VER/julia-$JULIA_VERSION-linux-x86_64.tar.gz"
  wget -nv $URL -O /tmp/julia.tar.gz # -nv means "not verbose"
  tar -x -f /tmp/julia.tar.gz -C /usr/local --strip-components 1
  rm /tmp/julia.tar.gz

  # Install Packages
  if [ "$COLAB_GPU" = "1" ]; then
      JULIA_PACKAGES="$JULIA_PACKAGES $JULIA_PACKAGES_IF_GPU"
  fi
  for PKG in `echo $JULIA_PACKAGES`; do
    echo "Installing Julia package $PKG..."
    julia -e 'using Pkg; pkg"add '$PKG'; precompile;"' &> /dev/null
  done

  # Install kernel and rename it to "julia"
  echo "Installing IJulia kernel..."
  julia -e 'using IJulia; IJulia.installkernel("julia", env=Dict(
      "JULIA_NUM_THREADS"=>"'"$JULIA_NUM_THREADS"'"))'
  KERNEL_DIR=`julia -e "using IJulia; print(IJulia.kerneldir())"`
  KERNEL_NAME=`ls -d "$KERNEL_DIR"/julia*`
  mv -f $KERNEL_NAME "$KERNEL_DIR"/julia  

  echo ''
  echo "Successfully installed `julia -v`!"
  echo "Please reload this page (press Ctrl+R, ⌘+R, or the F5 key) then"
  echo "jump to the 'Checking the Installation' section."
fi

Installing Julia 1.6.0 on the current Colab Runtime...
2021-12-07 05:40:55 URL:https://storage.googleapis.com/julialang2/bin/linux/x64/1.6/julia-1.6.0-linux-x86_64.tar.gz [112838927/112838927] -> "/tmp/julia.tar.gz" [1]
Installing Julia package IJulia...
Installing IJulia kernel...
[ Info: Installing julia kernelspec in /root/.local/share/jupyter/kernels/julia-1.6

Please reload this page (press Ctrl+R, ⌘+R, or the F5 key) then
jump to the 'Checking the Installation' section.


## Checking the Installation
The `versioninfo()` function should print your Julia version and some other info about the system (if you ever ask for help or file an issue about Julia, you should always provide this information).

In [1]:
versioninfo()

Julia Version 1.6.0
Commit f9720dc2eb (2021-03-24 12:55 UTC)
Platform Info:
  OS: Linux (x86_64-pc-linux-gnu)
  CPU: Intel(R) Xeon(R) CPU @ 2.20GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-11.0.1 (ORCJIT, broadwell)
Environment:
  JULIA_NUM_THREADS = 4


# Imports

In [31]:
using Pkg
Pkg.add("Distances")

    Updating registry at `~/.julia/registries/General`
   Resolving package versions...
   Installed Distances ─ v0.10.7
   Installed StatsAPI ── v1.1.0
    Updating `~/.julia/environments/v1.6/Project.toml`
  [b4f34e82] + Distances v0.10.7
    Updating `~/.julia/environments/v1.6/Manifest.toml`
  [b4f34e82] + Distances v0.10.7
  [82ae8749] + StatsAPI v1.1.0
  [37e2e46d] + LinearAlgebra
  [2f01184e] + SparseArrays
  [10745b16] + Statistics
Precompiling project...
  ✓ StatsAPI
  ✓ Distances
2 dependencies successfully precompiled in 3 seconds (15 already precompiled)


In [32]:
using LinearAlgebra
using Statistics
using Dates
using Distances

# How to get all the dates corresponding to the month of July 2016?

In [3]:
dr = Date("2016-07-01"):Day(1):Date("2016-07-31")

Date("2016-07-01"):Day(1):Date("2016-07-31")

In [4]:
# The above is probably all you need but if if do need to get the dates individually then this works
dr = [Date("2016-07-01"):Day(1):Date("2016-07-31");]

31-element Vector{Date}:
 2016-07-01
 2016-07-02
 2016-07-03
 2016-07-04
 2016-07-05
 2016-07-06
 2016-07-07
 2016-07-08
 2016-07-09
 2016-07-10
 2016-07-11
 2016-07-12
 2016-07-13
 ⋮
 2016-07-20
 2016-07-21
 2016-07-22
 2016-07-23
 2016-07-24
 2016-07-25
 2016-07-26
 2016-07-27
 2016-07-28
 2016-07-29
 2016-07-30
 2016-07-31

# How to compute ((A+B)*(-A/2)) in place (without copy)?

In [5]:
A = ones(3)
B = 2ones(3)

(A + B) .* (-A / 2)

3-element Vector{Float64}:
 -1.5
 -1.5
 -1.5

# Extract the integer part of a random array of positive numbers?

In [6]:
Z = rand(0:0.01:100,20)
make_int(x) = trunc(Int,x)
make_int.(Z)

20-element Vector{Int64}:
 59
 30
  4
 88
  3
 51
 32
 95
 69
 23
 24
 14
 86
  9
 94
 29
  8
 72
 52
 79

# Create a 5x5 matrix with row values ranging from 0 to 4 

In [7]:
A = fill([0:4;], 5)
# This is the solution if you want a array of arrays

5-element Vector{Vector{Int64}}:
 [0, 1, 2, 3, 4]
 [0, 1, 2, 3, 4]
 [0, 1, 2, 3, 4]
 [0, 1, 2, 3, 4]
 [0, 1, 2, 3, 4]

In [8]:
# The solution for this particular problem
A = [0 1 2 3 4]
repeat(A, 5)

5×5 Matrix{Int64}:
 0  1  2  3  4
 0  1  2  3  4
 0  1  2  3  4
 0  1  2  3  4
 0  1  2  3  4

In [9]:
# This is a more generic solution but slightly more involved
repeat(reshape([0:4;], 1, :), 5)
# Lets break down as so 
# * [0:4;] creates a vector [0,1,2,3,4], by default Julia makes a column vector
# * reshape([0:4;], 1, :) converts to 1x5 row vector from the column vector
# * The repeat function basically copies it five times

5×5 Matrix{Int64}:
 0  1  2  3  4
 0  1  2  3  4
 0  1  2  3  4
 0  1  2  3  4
 0  1  2  3  4

# Create a vector of size 10 with values ranging from 0 to 1, both excluded


In [10]:
Z = [rand(Float64) for i in 1:10]
# Most of the time this will work. I don't really see the need to actually exclude 0 as well from this code but if you do, then

10-element Vector{Float64}:
 0.7165166080997047
 0.2696561325638622
 0.8831933496131028
 0.6915696847598534
 0.4904262840306819
 0.07250199202949714
 0.3731427785645258
 0.5531284289109053
 0.7333200435534386
 0.21130312349370994

In [11]:
Z = [prevfloat(1.0) * (1 - rand(Float64)) for i in 1:10]
# Again its pretty extremely unlikely you'd actually need it but if you do, then you please read this discussion as well before going in https://discourse.julialang.org/t/how-to-create-a-random-uniform-distribution-between-but-excluding-0-and-10

10-element Vector{Float64}:
 0.03800248075192369
 0.5522688556693326
 0.7735801634673164
 0.9611256141528276
 0.24997884703532255
 0.49434921637825163
 0.6410719729579945
 0.2133674267459684
 0.841468068666206
 0.8955416655565974

# Create a random vector of size 10 and sort it

In [12]:
Z = rand(Int,10)
sort!(Z)

10-element Vector{Int64}:
 -8105931568320287737
 -5549860894228175336
 -2112434236849380150
   874346798032584509
  2079082836835675050
  3830037054528337295
  3893757251933956159
  5160360531292831239
  6184719450628982333
  8511439861048190173

# Consider two random array A and B, check if they are equal

In [13]:
A = rand(1:5, 10)
B = rand(1:5, 10)

# To check if the arrays are completely equal
A == B

false

In [14]:
A = rand(1:5, 10)
B = rand(1:5, 10)
A,B

([4, 3, 1, 2, 4, 3, 5, 1, 2, 4], [3, 2, 1, 1, 5, 4, 1, 2, 3, 4])

In [15]:
# To check if the elements in the arrays are equal
A .== B

10-element BitVector:
 0
 0
 1
 0
 0
 0
 0
 0
 0
 1

# Consider a random 10x2 matrix representing cartesian coordinates, convert them to polar coordinates


In [16]:
struct cartesian_point
    X::Float32
    Y::Float32
end
points = [cartesian_point(rand(-900:0.01:900), rand(-900:0.01:900)) for _ = 1:10]

10-element Vector{cartesian_point}:
 cartesian_point(-733.36f0, -540.92f0)
 cartesian_point(-303.5f0, -486.86f0)
 cartesian_point(-172.14f0, 71.91f0)
 cartesian_point(-801.91f0, -816.82f0)
 cartesian_point(577.89f0, 837.57f0)
 cartesian_point(258.95f0, 631.2f0)
 cartesian_point(-753.47f0, -28.27f0)
 cartesian_point(-181.49f0, -335.75f0)
 cartesian_point(873.57f0, -468.53f0)
 cartesian_point(-408.81f0, -3.4f0)

In [17]:
points = [cartesian_point(rand(), rand()) for _ = 1:10]
Xpoints = [point.X for point in points]
Ypoints = [point.Y for point in points]

10-element Vector{Float32}:
 0.59284914
 0.87740934
 0.5979325
 0.86323935
 0.41968682
 0.926866
 0.62350166
 0.9527373
 0.5619024
 0.3643342

In [18]:
radians(x, y) = sqrt(x^2 + y^2)
theta(x, y) = atan(y, x)

theta (generic function with 1 method)

In [19]:
radians.(Xpoints, Ypoints)
theta.(Xpoints, Ypoints)

10-element Vector{Float32}:
 0.8255184
 1.1365886
 0.56720984
 1.5537957
 0.5093731
 1.1861624
 0.82003444
 1.4892243
 0.65683484
 0.5819314

# Create random vector of size 10 and replace the maximum value by 0


In [20]:
Z = rand(Int8, 10)
print(Z)
Z[argmax(Z)]

Int8[10, -125, -84, 48, -48, -95, 85, 42, -86, -42]

85

# Create a structured array with x and y coordinates covering the [0,1]x[0,1] area


In [21]:
Z = (X = collect(LinRange(0, 1, 5)), Y = collect(LinRange(0, 1, 5)))

(X = [0.0, 0.25, 0.5, 0.75, 1.0], Y = [0.0, 0.25, 0.5, 0.75, 1.0])

# Given two arrays, X and Y, construct the Cauchy matrix C (Cᵢⱼ = 1 ÷ (xᵢ - yⱼ))

In [22]:
X = rand(Float32, 10)
Y = rand(Float32, 10)

10-element Vector{Float32}:
 0.91190946
 0.07268238
 0.045708776
 0.19349325
 0.80093217
 0.46550834
 0.30717564
 0.9135554
 0.6839801
 0.3817637

In [23]:
f(i, j) = 1 ÷ (i - j)
C = (f(x, y) for x in X, y in Y)
det(collect(C))

-1.5848783f7

# How to find the closest value (to a given scalar) in a vector?

In [25]:
A = rand(Int8, 10)
print(A)
findnearest(A, x) = argmin(abs.(A .- x))

Int8[-3, 91, 105, 2, -114, 95, 52, 85, 106, 11]

findnearest (generic function with 1 method)

In [27]:
A[findnearest(A,0)]

2

# Consider a random vector with shape (100,2) representing coordinates, find point by point distances

In [40]:
A = rand(Int8, (10, 2))
B = rand(Int8, (10, 2))

0.0

In [42]:
pairwise(Euclidean(), A, B, dims=1)

10×10 Matrix{Float64}:
  0.0      0.0       0.0      …  14.8661  0.0       0.0      0.0
  0.0     21.2132    0.0          0.0     0.0       0.0      0.0
  0.0     17.0      18.4391      17.5499  9.16515  13.0384   8.06226
 15.6205  10.8167   10.0         12.1655  0.0       0.0      0.0
 11.8743   8.60233   0.0          0.0     0.0       8.544    0.0
  0.0     15.2971    9.43398  …   0.0     0.0       0.0      0.0
  0.0      8.544     0.0          0.0     0.0       6.48074  0.0
 16.2788   0.0      13.3041       5.0     7.0       0.0      0.0
 18.1108   0.0       0.0          0.0     0.0       5.83095  0.0
  0.0     13.8924    2.82843     10.198   0.0       0.0      0.0

# How to convert a float (32 bits) array into an integer (32 bits) in place

In [43]:
A = rand(Float32, 10)
trunc.(Int, A)

10-element Vector{Int64}:
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0